In [ ]:
events = run.run(initial_state, time_steps=1000)
events = processing.postprocessing(events)

In [ ]:
pu.plot(events, pool='stableswap', prop='liquidity')
pu.plot(events, agent='Arbitrageur', prop='holdings', key=['S1', 'S2'])

In [ ]:
impermanent_loss = [
    (1 - event.pools['stableswap'].cash_out(event.agents['LP'], event.external_market)
    / sum(event.agents['non LP'].holdings.values())) * 100
    for event in events
]
total_supply = [
    sum([
        event.pools['stableswap'].liquidity['S1'], 
        event.agents['Arbitrageur'].holdings['S1'], 
        event.agents['LP'].holdings['S1'], 
        event.agents['non LP'].holdings['S1']
    ]) for event in events
]


In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(total_supply, impermanent_loss)
plt.title

In [ ]:
events[2].agents['LP'].holdings

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(total_supply, [event.pools['stableswap'].cash_out(event.agents['LP'], event.external_market) for event in events])
plt.plot(total_supply, [sum(event.agents['non LP'].holdings.values()) for event in events])

In [ ]:
(
    events[-1].pools['stableswap'].liquidity['S1']
    + events[-1].agents['LP'].holdings['S1']
    + events[-1].agents['non LP'].holdings['S1']
    + events[-1].agents['Arbitrageur'].holdings['S1']
)

In [ ]:
events[-1].agents['non LP'].holdings['S1']